In [1]:
%%writefile simple.yml
shipment:
  origins:
  - Netherlands
  - Mexico
  flowers:
  - Hyacinthus
  - Rosa
  - Gerbera
  boxes:
    min: 1
    max: 50
pest:
  infestation_rate:
    distribution: beta
    parameters:
    - 4
    - 60
  arrangement: random_box
  random_box:
    probability: 0.2
    ratio: 0.5
inspection:
  unit: stems
  within_box_pct: 1.0
  sample_strategy: percentage
  percentage:
    proportion: 0.02
    min_boxes: 1
  hypergeometric:
    detection_level: 0.05
    confidence_level: 0.95
    min_boxes: 1
  fixed_n: 10
  selection_strategy: random
  end_strategy: to_completion
ports:
  - NY JFK CBP
  - FL Miami Air CBP
stems_per_box:
  default: 10

Overwriting simple.yml


In [2]:
!python3 -m pathways --num-simulations 1 --num-shipments 10 --config-file simple.yml --pretty --seed=1

━━ Shipment ━━ Boxes: 17 ━━ Stems: 170 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 🐛 | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
━━ Shipment ━━ Boxes: 28 ━━ Stems: 280 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ | ✿ ✿ 